In [3]:
import os
import pandas as pd

# Definindo os caminhos para as pastas
diretorio_fake =  ''
diretorio_real = ''

# Lista para armazenar os dados
dados = []

# Lendo arquivos da pasta fake
for arquivo in os.listdir(diretorio_fake):
    caminho_completo = os.path.join(diretorio_fake, arquivo)
    dados.append([caminho_completo, 1])  # 1 para fake

# Lendo arquivos da pasta real
for arquivo in os.listdir(diretorio_real):
    caminho_completo = os.path.join(diretorio_real, arquivo)
    dados.append([caminho_completo, 0])  # 0 para real

# Criando o DataFrame
df = pd.DataFrame(dados, columns=['wav_file', 'Label'])

In [ ]:
# import tqdm
# import torchaudio
# import tqdm
# from collections import namedtuple

# Info = namedtuple("Info", ["length", "sample_rate", "channels"])

# def get_audio_info(path: str):
#     info = torchaudio.info(path)
#     if hasattr(info, 'num_frames'):
#         return Info(info.num_frames, info.sample_rate, info.num_channels)
#     else:
#         siginfo = info[0]
#         return Info(siginfo.length // siginfo.channels, siginfo.rate, siginfo.channels)


# def get_total_dataset_length(df):
#     length = []
#     srs = []
#     channels = []

#     for file_path in tqdm.tqdm(df['wav_file']):
#         audio_info = get_audio_info(file_path)
#         srs.append(audio_info.sample_rate)
#         channels.append(audio_info.channels)
#         length.append(audio_info.length / audio_info.sample_rate)

#     print()
#     print("-"*50)
#     print(f"Min audio length (in seconds): {min(length)} | Max audio length (in seconds): {max(length)}")
#     print(f"Mean audio length (in seconds): {np.mean(length)} | Median: {np.median(length)} | Std: {np.std(length)}")
#     print(f"Total amount of data (in minutes): {np.sum(length) / 60}")
#     print('-'*50)
#     print(f"Different sample rates in audios in the dataset: {set(srs)}")
#     print(f"Different number of channels in the audios in the dataset: {set(channels)}")

# get_total_dataset_length(df)

100%|██████████| 5347/5347 [16:03<00:00,  5.55it/s]


--------------------------------------------------
Min audio length (in seconds): 0.07395833333333333 | Max audio length (in seconds): 30.037333333333333
Mean audio length (in seconds): 4.379792847473757 | Median: 3.34 | Std: 3.5586364891184306
Total amount of data (in minutes): 390.3125392573696
--------------------------------------------------
Different sample rates in audios in the dataset: {24000, 48000, 8000, 16000, 44100}
Different number of channels in the audios in the dataset: {1, 2}


As taxas de amostragem são 16000Hz e 44100Hz. Vamos manter a taxa em 16000Hz.

## Extração de features

Existem algumas features que podem ser úteis:



1. Zero Crossing Rate: A taxa de mudanças de sinal do sinal durante a duração de um quadro específico.
2. Energy: A soma dos quadrados dos valores do sinal, normalizada pelo respectivo comprimento do quadro.
3. Entropy of Energy: A entropia das energias normalizadas dos subquadros. Pode ser interpretado como uma medida de mudanças abruptas.
4. Spectral Centroid: O centro de gravidade do espectro.
5. Spectral Spread: O segundo momento central do espectro.
6. Spectral Entropy: Entropia das energias espectrais normalizadas para um conjunto de subquadros.
7. Spectral Flux: A diferença quadrática entre as magnitudes normalizadas dos espectros dos dois quadros sucessivos.
8. Spectral Rolloff: A frequência abaixo da qual 90% da distribuição de magnitude do espectro está concentrada.
9. Os coeficientes cepstrais de frequência Mel dos MFCCs formam uma representação cepstral onde as bandas de frequência não são lineares, mas distribuídas de acordo com a escala mel.

Features escolhidas:


1.   MFCCs: Capturam informações timbrais que são cruciais para diferenciar emoções sutis.
2.  Energia:  energia de um sinal de voz, que mede sua intensidade, varia significativamente com diferentes emoções. Emoções como raiva e excitação tendem a ser expressas com maior energia (mais alto e mais intenso), enquanto tristeza e calma são frequentemente expressas com menor energia.
3. Pitch (tom): pitch, ou a frequência fundamental da voz, é um indicador primário do estado emocional do falante. Emoções excitadas como alegria ou raiva geralmente resultam em um aumento do pitch, enquanto emoções como tristeza ou desânimo podem levar a um pitch mais baixo.



In [ ]:
from tqdm.auto import tqdm
from torchaudio.transforms import Resample
import torch

tqdm.pandas()

resampled_count = 0

def extract_features(file_path, target_sample_rate=16000):
    global resampled_count

    audio, sample_rate = librosa.load(file_path, sr=None)
    audio = torch.tensor(audio)

    if sample_rate != target_sample_rate:
        resampled_count += 1
        resampler = Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        audio = resampler(audio.unsqueeze(0)).squeeze(0)
        sample_rate = target_sample_rate

    audio = audio.numpy()

    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
    mfccs_mean = np.mean(mfccs, axis=1)
    energy = np.sum(np.square(audio)) / len(audio)
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sample_rate)
    pitch = np.mean(pitches[pitches > 0]) if np.any(pitches > 0) else 0

    features = np.concatenate((mfccs_mean, [energy, pitch]))
    return features

df['features'] = df['wav_file'].progress_apply(extract_features)

print(f"Total de áudios resampleados: {resampled_count}")

feature_names = [f'mfcc_{i}' for i in range(20)] + ['energy', 'pitch']
features_df = pd.DataFrame(df['features'].tolist(), columns=feature_names)
final_df = pd.concat([df[['Label']], features_df], axis=1)

In [6]:
final_df

,Label,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,energy,pitch
0,1,188.146439,50.862251,11.253999,14.367369,-4.731651,11.239244,-10.804421,1.751063,-3.267571,...,-12.275487,-6.476327,-4.015410,0.591770,-2.020293,-9.299771,-3.956975,-8.307939,0.858367,1751.110107
1,1,-390.428009,110.153557,-33.775627,33.273331,2.238101,-20.178146,6.072151,-20.428488,-7.699133,...,-5.098675,-7.042708,4.334699,-9.657085,-2.743689,1.458176,-9.009430,0.183653,0.007073,1819.989014
2,1,-328.147217,88.179794,-6.587357,29.497618,-9.674681,-1.494627,-13.455149,-6.576401,-10.464543,...,-8.342735,0.395463,-6.992785,-4.529456,-5.165907,-5.638328,-5.840557,-3.393305,0.002072,1684.312866
3,1,-324.112610,91.268105,-4.672131,44.326645,-13.536561,10.045351,-19.289839,-3.611017,-3.036355,...,-3.720351,-14.839140,-3.908122,-14.414676,-7.742877,-12.475356,-8.540500,-9.365683,0.009628,1531.818848
4,1,-212.305511,106.320839,-31.366352,29.945105,-20.282455,2.338663,-20.500937,-4.557822,-6.926822,...,-5.472649,2.398657,-5.462188,-9.760441,-1.431723,-12.167992,-4.246058,-5.164224,0.007295,1337.947510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5342,0,-324.964630,83.862045,-11.241543,37.746269,-16.604425,-1.976251,-11.311910,3.630836,-9.320843,...,-4.508473,-13.305090,-5.981932,-5.786707,-9.646820,-11.787356,-5.955360,-11.646958,0.012816,1212.517578
5343,0,-204.733719,120.581764,-37.977562,39.463825,-35.014137,21.631462,-28.614771,5.773151,-3.124882,...,4.126869,-17.018272,4.023340,-7.961537,-5.251965,-9.942708,-3.165019,-6.489861,0.013607,1434.389160
5344,0,-370.050995,87.425377,-9.646264,38.022842,1.096222,13.898228,-12.397937,-11.221161,-1.519010,...,-10.143262,-3.418483,-4.163138,-12.845467,1.446856,-8.565943,-6.969484,-6.190107,0.009333,1383.394653
5345,0,-321.963806,132.376190,-33.803699,32.883087,-5.166338,-15.684356,8.962790,-20.756620,7.608210,...,-6.004715,-8.815777,2.738862,-9.888856,2.283211,0.852312,-7.879587,2.578317,0.010025,1451.709595


## Modelagem/Resultados

## Teste

In [7]:
import os
import pandas as pd

# Definindo o caminho para a pasta de teste
diretorio_teste = ''

# Lista para armazenar os dados
dados = []

# Lendo arquivos da pasta de teste
for arquivo in os.listdir(diretorio_teste):
    caminho_completo = os.path.join(diretorio_teste, arquivo)
    dados.append([caminho_completo])  # Apenas o caminho do arquivo

# Criando o DataFrame
df_test = pd.DataFrame(dados, columns=['wav_file'])

In [ ]:
df_test['features'] = df_test['wav_file'].progress_apply(extract_features)
feature_names = [f'mfcc_{i}' for i in range(20)] + ['energy', 'pitch']
features_df = pd.DataFrame(df_test['features'].tolist(), columns=feature_names)
#final_df_test = pd.concat([df_test[['label']], features_df], axis=1)

In [9]:
features_df

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,energy,pitch
0,-292.180450,84.900177,-39.799118,30.045853,-37.579548,3.481106,-20.662188,-20.180958,-1.217206,-14.285432,...,-4.951134,-15.851294,-9.446732,-9.215483,-12.739226,-6.832196,-10.144959,-12.420197,0.016293,1649.940552
1,-273.587860,142.820831,-57.152752,49.145008,-9.923056,-38.471939,9.299982,-30.570486,-11.380567,1.513258,...,-9.431334,-17.003910,-1.888696,-15.459139,-4.384775,-3.141651,-13.139039,-6.850340,0.021116,1612.341675
2,-341.391052,72.565987,17.846224,41.163021,-0.319392,-2.003624,-11.028720,-8.386998,-7.089276,-7.111572,...,-1.462955,-1.479210,-7.953369,-3.841994,-2.587147,-10.460544,-4.850620,-2.283415,0.003151,1641.924438
3,-318.563629,86.191948,15.649541,27.988272,1.546294,-2.046189,-14.017217,-10.191701,-7.772046,-9.560753,...,-3.413128,-4.386826,-7.100698,-7.049280,-2.762381,-11.455525,-7.919273,-2.632838,0.002928,1352.703735
4,-204.606064,35.595303,-15.523376,31.840593,-8.696500,-6.327109,-21.348137,-7.703871,-8.654745,-4.613372,...,-10.302103,6.104741,-11.450482,2.363475,-6.229240,-3.476623,-2.013077,-4.505461,0.003513,2045.835083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,-323.126526,94.426987,18.777361,24.631571,8.893686,-4.563370,-3.650044,-7.216611,-3.597112,-6.740106,...,-11.650814,-1.107422,-4.047962,-6.456823,-3.989221,-3.560990,-8.326303,-4.193917,0.002714,1442.992432
1926,-334.062469,87.060204,-14.084995,5.044766,-24.027855,1.310656,-15.577295,-11.149854,-4.509669,-3.527222,...,-2.785758,-3.000867,-7.199112,-6.947437,-6.197103,-6.595174,-4.261516,-5.074823,0.002825,1511.718994
1927,-354.270874,101.923065,-22.785999,19.332554,2.705379,-0.471154,7.405931,-18.454067,0.892806,2.003413,...,-3.876410,-1.194842,4.123882,-10.024743,-1.545583,-0.687144,-8.052211,-2.001942,0.015005,1465.948730
1928,-366.100861,91.043755,17.326536,23.011688,12.371262,5.975401,-2.488668,0.266167,-2.265783,-2.327530,...,1.802845,3.654872,8.243746,-0.054925,8.680608,3.289943,-0.040585,3.009249,0.003842,1705.061401


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


def train_and_evaluate(X, y, test_size=0.05, random_state=42, model_type='logistic'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    if model_type == 'logistic':
        model = LogisticRegression(max_iter=1000)
    elif model_type == 'svm':
        model = SVC(probability=True, kernel ='rbf', max_iter=-1)
    elif model_type == 'xgboost':
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    elif model_type == 'knn':
        model = KNeighborsClassifier()
    elif model_type == 'random_forest':
        model = RandomForestClassifier()
    elif model_type == 'mlp':
        model = MLPClassifier(max_iter=1000)

    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    probabilities = None
    if hasattr(model, 'predict_proba'):
        probabilities = model.predict_proba(X_test_scaled)[:, 1]

    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return scaler, model, probabilities

# Treinando o modelo XGBoost e obtendo o scaler
X = final_df.drop('Label', axis=1)
y = final_df['Label']
scaler, model_xgb, _ = train_and_evaluate(X, y, model_type='svm')

# Função para prever probabilidades
def predict_probabilities(df_test, scaler, model):
    X_test_scaled = scaler.transform(df_test)
    probabilities = model.predict_proba(X_test_scaled)[:, 1]
    return probabilities

# Suponha que features_df é um DataFrame com novos dados de teste
probabilities_class_1 = predict_probabilities(features_df, scaler, model_xgb)
print("Probabilities for class 1 on new test data:", probabilities_class_1)

Accuracy: 0.8992537313432836
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.85      0.85        91
           1       0.92      0.93      0.92       177

    accuracy                           0.90       268
   macro avg       0.89      0.89      0.89       268
weighted avg       0.90      0.90      0.90       268

Probabilities for class 1 on new test data: [0.96115966 0.98311179 0.91625816 ... 0.01402434 0.70435473 0.9638599 ]


In [11]:
len(probabilities_class_1)

1930

In [12]:
probabilities_class_1

array([0.96115966, 0.98311179, 0.91625816, ..., 0.01402434, 0.70435473,
       0.9638599 ])

In [13]:
df_sub = pd.read_csv('/content/drive/MyDrive/7 PERIODO/PAV/deepfake-detection-ufg/sample_solution.csv')

In [14]:
df_sub

,id,fake_prob
0,050d5129-cd35-4ae5-a3a0-c32c4aac2fab.mp3,0.909789
1,05f4d86f-9ec2-46ab-a9f7-c764e819f8ce.mp3,0.948911
2,0920c8c8-8889-45f5-ba3c-b89f90f94ae7.mp3,0.963187
3,096086d2-d592-4931-8bab-aaef63417477.mp3,0.993231
4,0ab8cbd5-d35b-4692-857e-94865f71ba9e.mp3,0.974185
...,...,...
1925,ce367c6d-c152-48c2-9be3-3689b86ebe65.mp3,0.515561
1926,83176a9a-b75e-4af6-aa4c-aee9ea92df6f.mp3,0.145915
1927,86157684-059a-4ca7-bf62-c9d9baa32092.mp3,0.998759
1928,80978467-7b6e-40c2-b26e-0d40d16df108.mp3,0.587040


In [15]:
df_sub.loc[df_sub['id'] == '7116129d-a23a-4a47-9643-ba1e5b4e8aad.mp3']

,id,fake_prob
1710,7116129d-a23a-4a47-9643-ba1e5b4e8aad.mp3,0.924258


In [16]:
df_test

,wav_file,features
0,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-292.1804504394531, 84.90017700195312, -39.79..."
1,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-273.5878601074219, 142.82083129882812, -57.1..."
2,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-341.39105224609375, 72.56598663330078, 17.84..."
3,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-318.5636291503906, 86.19194793701172, 15.649..."
4,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-204.60606384277344, 35.59530258178711, -15.5..."
...,...,...
1925,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-323.12652587890625, 94.42698669433594, 18.77..."
1926,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-334.0624694824219, 87.0602035522461, -14.084..."
1927,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-354.2708740234375, 101.92306518554688, -22.7..."
1928,/content/drive/MyDrive/7 PERIODO/PAV/deepfake-...,"[-366.1008605957031, 91.04375457763672, 17.326..."


In [ ]:
# df_test['id'] = df_test['id'].str.split('/').str[-1]

In [17]:
df_test['wav_file'] = df_test['wav_file'].str.split('/').str[-1]

In [18]:
df_test['fake_prob'] = probabilities_class_1

In [19]:
df_test = df_test.rename(columns={'wav_file': 'id'})

In [20]:
df_sub = df_sub.drop('fake_prob', axis=1)

In [21]:
df_merged = df_sub.merge(df_test, on='id', how='left')

In [22]:
df_merged = df_merged.drop('features', axis = 1)

In [23]:
df_merged.to_csv('SVM', index=False)